## Library
---

In [16]:
import os
import time
import numpy as np


## Algorithm
---

In [2]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [65]:
# fcc and bcc using numpy!
def create_fcc_lattice(height, width, depth):
    # Create a 3D matrix filled with zeros
    lattice = np.zeros((height, width, depth), dtype=int)

    # Fill the matrix with ones at the positions of the atoms in the FCC lattice
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                if (x % 2 == 0 and y % 2 == 0 and z % 2 == 0) or \
                   (x % 2 == 1 and y % 2 == 1 and z % 2 == 0) or \
                   (x % 2 == 1 and y % 2 == 0 and z % 2 == 1) or \
                   (x % 2 == 0 and y % 2 == 1 and z % 2 == 1):
                    lattice[z, x, y] = 1
    
    # add vaccum layers
    vaccum = np.zeros((1, width, depth))
    lattice = np.concatenate((vaccum, lattice, vaccum), axis=0)

    return lattice

def create_bcc_lattice(height, width, depth):
    # Create a 3D matrix filled with zeros
    lattice = np.zeros((height, width, depth))

    # Fill the matrix with ones at the positions of the atoms in the BCC lattice
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                if (x % 2 == 0 and y % 2 == 0 and z % 2 == 0) or \
                   (x % 2 == 1 and y % 2 == 1 and z % 2 == 1):
                    lattice[z, x, y] = 1

    # add vaccum layers
    vaccum = np.zeros((1, width, depth))
    lattice = np.concatenate((vaccum, lattice, vaccum), axis=0)

    return lattice

def create_simple_cubic_lattice(height, width, depth):
    lattice = np.ones((height+2, width, depth), dtype=int)
    lattice[0, :, :] = 0
    lattice[-1, :, :] = 0

    return lattice


In [16]:
# lattice to xyz
def save_matrix_as_xyz(matrix, atom, output_filename):
    with open(output_filename, 'a') as output_file:
        num_atoms = int(np.sum(matrix))  # Calculate the number of Cu atoms (1s)

        # Write the number of atoms as the first line in the XYZ file
        output_file.write(f"{num_atoms}\n\n")

        # Iterate through the matrix to write the atom positions
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                for k in range(matrix.shape[2]):
                    if matrix[i, j, k] == 1:  # Cu atom
                        output_file.write(f"{atom} {i} {j} {k}\n")

In [79]:
test_fcc = create_fcc_lattice(3, 10, 10)
test_bcc = create_bcc_lattice(10, 10, 10)

In [80]:
'''
find nearest atoms around the site
'''
def get_atoms_around_site_fcc(lattice, z, x, y):
    height, width, depth = lattice.shape
    current_pos = np.array([z, x, y])
    nearest_atoms = np.array([[1, 1, 0], [-1, 1, 0], [1, -1, 0], [-1, -1, 0],
                              [1, 0, 1], [-1, 0, 1], [1, 0, -1], [-1, 0, -1],
                              [0, 1, 1], [0, -1, 1], [0, 1, -1], [0, -1, -1]])
    nearest_atoms_pos = nearest_atoms + current_pos
    result = []

    for pos in nearest_atoms_pos:
        if 0 <= pos[0] <= height+1 and lattice[*pos.tolist()] == 1:
            result.append(pos)
    return result
    
def get_atoms_around_site_bcc(lattice, z, x, y):
    height, width, depth = lattice.shape
    current_pos = np.array([z, x, y])
    nearest_atoms = np.array([[1, 1, 1], [-1, 1, 1], [1, -1, 1], [1, 1, -1],
                              [-1, -1, -1], [-1, -1, 1], [-1, 1, -1], [1, -1, -1]])
    
    nearest_atoms_pos = nearest_atoms + current_pos
    result = []
    for pos in nearest_atoms_pos:
        if 0 <= pos[0] <= height+1 and lattice[*pos.tolist()] == 1:
            result.append(pos)
    return result

In [81]:
# find candidate
def find_candidate_fcc(lattice):
    global e_a
    global diffusion_rate
    height, width, depth = lattice.shape
    candidate_table = []
    diffusion_table = []
    motion_table = []
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                # find vacancy
                if lattice[z, x, y] == 0:
                    up, down, left, forward, right, back = get_atoms_around_site(lattice, z, x, y)
                    neighbor = left+up+right+down+forward+back
                    if neighbor:
                        '''
                        motion table
                        1 : up     2 : down     3 : left       4 : forward      5 : right       6 : back
                        '''
                        if up:
                            neighbor_of_up = get_atoms_around_site(lattice, z-1, x, y, True)
                            candidate_table.append((z-1, x, y))
                            motion_table.append(2)
                            diffusion_table.append(diffusion_rate[neighbor_of_up, neighbor-1])
                        if down:
                            neighbor_of_down = get_atoms_around_site(lattice, z+1, x, y, True)
                            candidate_table.append((z+1, x, y))
                            motion_table.append(1)
                            diffusion_table.append(diffusion_rate[neighbor_of_down, neighbor-1]) 
                        if left:
                            neighbor_of_left = get_atoms_around_site(lattice, z, x, (y-1)%depth, True)
                            candidate_table.append((z, x, (y-1)%depth))
                            motion_table.append(5)
                            diffusion_table.append(diffusion_rate[neighbor_of_left, neighbor-1])
                        if forward:
                            neighbor_of_forward = get_atoms_around_site(lattice, z, (x-1)%width, y, True)
                            candidate_table.append((z, (x-1)%width, y))
                            motion_table.append(6)
                            diffusion_table.append(diffusion_rate[neighbor_of_forward, neighbor-1]) 
                        if right:
                            neighbor_of_right = get_atoms_around_site(lattice, z, x, (y+1)%depth, True)
                            candidate_table.append((z, x, (y+1)%depth))
                            motion_table.append(3)
                            diffusion_table.append(diffusion_rate[neighbor_of_right, neighbor-1])
                        if back:
                            neighbor_of_back = get_atoms_around_site(lattice, z, (x+1)%width, y, True)
                            candidate_table.append((z, (x+1)%width, y))
                            motion_table.append(4)
                            diffusion_table.append(diffusion_rate[neighbor_of_back, neighbor-1]) 
    
    return candidate_table, diffusion_table, motion_table


[array([4, 1, 1]),
 array([2, 1, 1]),
 array([ 4, -1,  1]),
 array([ 4,  1, -1]),
 array([ 2, -1, -1]),
 array([ 2, -1,  1]),
 array([ 2,  1, -1]),
 array([ 4, -1, -1])]

In [69]:
test[2, -1, 0]

1.0

## Parameter setting
---

In [6]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
fcc -> 12 nearest neighbors
'''
bond_energy = 2.07 
temperature = 500
e0 = 0.1
num_closest_neighbors = 12

# site energy, e_(bond number)
e_site = np.zeros(num_closest_neighbors)
for i in range(num_closest_neighbors):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   

In [ ]:
# value check
# print(e_site)
# print('-----------------')
# # print(e_a)
# print('-----------------')
# print(diffusion_rate)

## Simulation
---

In [8]:
# parameter tuning
time_elapsed = 0
save_file_name = 'ovito/test_hcp_single.xyz'
steps = 1000
width = 10
depth = 10
height = 3
vacancy_idx = 100

atom_type_fcc = 'Cu'
lattice_param_fcc = 3.6
atom_type_bcc = 'Fe'
lattice_param_bcc = 2.87
atom_type_hcp = 'Ti'
lattice_param_hcp = 2.94

# create lattice
lattice_fcc = create_fcc_lattice(atom_type_fcc, lattice_param_fcc, [width, depth, height])
lattice_bcc = create_bcc_lattice(atom_type_bcc, lattice_param_bcc, [width, depth, height])
lattice_hcp = create_hcp_lattice(atom_type_hcp, lattice_param_hcp, [width, depth, height])

# make vacancy
# lattice[vacancy_idx].symbol = 'X'
# prev_vacncy_z = None
# vacancy_z = lattice[vacancy_idx].position[2]
# write(save_file_name, lattice)



/tmp/ipykernel_6590/2467453919.py:30: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  fcc_lattice *= repetitions
/tmp/ipykernel_6590/2467453919.py:60: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  bcc_lattice *= repetitions
/tmp/ipykernel_6590/2467453919.py:91: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  hcp_lattice *= repetitions


In [ ]:
# start simulation
for i in range(1, steps+1):
    prev_vacncy_z = vacancy_z
    vacancy_z = diffuse_one_step(lattice)
    # print(prev_vacncy_z, vacancy_z)
    if prev_vacncy_z != vacancy_z:
        write(save_file_name, lattice, append=True)
    elif i % 10 == 0:
        write(save_file_name, lattice, append=True)
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed in simul: {time_elapsed} s')

## Test area
---